실험설계는 사실상 모든 응용 연구 분야에서 통계분석의 토대가 된다. 실험설계는 어떤 가설을 확인하거나 기각하기 위한 목표를 갖고 있다. 

이 장에서는 전통적인 실험설계에 대해 알아보고 데이터 과학에도 적용되는 몇몇 어려움에 대해 논의한다. 또한 통계적 추론에서 자주 인용되는 일부 개념들을 다루고 데이터 과학에서의 의미와 관련성(또는 무관련성)을 설명한다.

통계적 유의성, t 검정, p 값 등에 대한 자료를 찾아보면 전형적인 통게적 추론이라는 ***파이프 라인*** 속에 있음을 알 수 있다. 실험은 가설을 검정하기 위해 설계되고 원하는 최종적인 결론을 도출할 수 있도록 설계된다. 그리고 데이터를 수집하고 분석한 다음 마침내 결론을 도출한다. 

[전형적인 통계 추론 과정]

1. 가설을 세운다.
2. 실험을 설계한다.
3. 데이터를 수집한다.
4. 추론 및 결론을 도출한다.

# 3-1 A/B 검정

A/B 검정은 두 가지 처리 방법, 제품, 절차 중 어느 쪽이 다른 쪽보다 더 우월하다는 것을 입증하기 위해 실험군을 두 그룹으로 나누어 진행하는 실험이다. 종종 두 가지 처리 방법 중 하나는 기준이 되는 기존 방법이거나 아예 아무런 처리도 적용하지 않는 방법이 된다. 이를 ***대조군***이라고 한다. 

[용어 정리]
- 처리(Treatment): 어떤 대상에 주어지는 특별한 환경이나 조건(약, 가격, 인터넷 뉴스 제목)
- 처리군(Treatment Group): 특정 처리에 노출된 대상들의 집합
- 대조군/대조그룹(Control Group): 어떤 처리도 하지 않은 대상들의 집단
- 임의화/랜덤화(Randomization): 처리를 적용할 대상을 임의로 결정하는 과정
- 대상(Subject): 처리를 적용할 개체 대상(유의어: 피실험자)
- 검정통계량(test Statistics): 처리 효과를 측정하기 위한 지표

[A/B검정 예시]
- 종자 발아가 어디에서 더 잘 되는지 알아보기 위해 두 가지 토양 처리를 검정한다.
- 암을 더 효과적으로 억제하는 두 가지 치료법을 검정한다.
- 두 가지 가격을 검정하여 더 많은 순이익을 산출하는 쪽을 결정한다.
- 두 개의 인터넷 뉴스 제목을 검정하여 더 많은 클릭을 생성하는 쪽을 결정한다.
- 두 개의 인터넷 광고를 검정하여 어느 것이 더 높은 전환율을 얻을지 판단한다.

제대로 된 A/B 검정에는 둘 중 어느 한쪽의 처리를 할당할 수 있는 대상이 주어진다. 이상적으로 피험자는 무작위로 어느 처리에 할당한다. 그러면 처리군 간의 차이는 다음 두 가지 이유 중 하나 때문이라고 할 수 있다.
- 다른 처리의 효과
- 어떤 대상이 어떤 처리에 배정될지에 대한 경우의 수 (즉 무작위로 배정한 결과 자연스럽게 더 좋은 결과를 보이는 대상들이 A 또는 B 한쪽에 집중됨)